In [ ]:
# !pip install -U langchain
!pip install langchain langchain-community langchain-core
!pip install langchain-openai openai
!pip install langchain-deepseek
!pip install deepseek-api python-dotenv

# Installing the OpenAI integration
#!pip install -U langchain-openai
# Installing the Anthropic integration
# !pip install -U langchain-anthropic

!pip show langchain-core

In [14]:
import os
from langchain.tools import tool, ToolRuntime
from langchain_openai import ChatOpenAI


# 设置 DeepSeek 的 API 密钥（LangChain-OpenAI 仍然会查找 OPENAI_API_KEY）
# 您可以尝试将 DEEPSEEK_API_KEY 的值赋给 OPENAI_API_KEY 环境变量
os.environ["OPENAI_API_KEY"] = os.environ.get("DEEPSEEK_API_KEY", "sk-943df854319e423ca178e68e4668ca5a") # 确保这个值是 DeepSeek 的 key

# 关键：指定 DeepSeek 的 API 基础 URL
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"

model = ChatOpenAI(
    model="deepseek-chat", # 使用 DeepSeek 的模型名称
    openai_api_base=DEEPSEEK_BASE_URL, # 指定 DeepSeek 的 URL
    temperature=0.7
)

In [17]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'text', 'text': "I'll get the current weather for San Francisco for you."}, {'type': 'tool_call', 'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_00_qtncGd1CizOohizAf1Uilez4'}]


step: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]


step: model
content: [{'type': 'text', 'text': "According to the weather information, it's always sunny in San Francisco!"}]




In [ ]:
from langchain.agents import create_agent


def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

In [18]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer  


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()  
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="custom"
):
    print(chunk)

Looking up data for city: San Francisco
Acquired data for city: San Francisco


In [21]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model,
    tools=[get_weather],
)

for stream_mode, chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["updates", "custom"]
):
    # print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")

content: {'model': {'messages': [AIMessage(content="I'll get the current weather for San Francisco for you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 153, 'total_tokens': 180, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 25}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '501431bf-8241-4cb7-a807-fac600cb0c23', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--aabcc28c-7637-4f65-a15b-434dc384e5d0-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_00_SNfiCSVaFwYBAyYhzGLUf4CV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 153, 'output_tokens': 27, 'total_tokens': 180, 'input_token_details': {'cache_read': 128}, 'output_token_details': {}})]}}


content: Looking u